In [20]:
import warnings

import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [30]:
asset_type = "building"

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
}

dataset_paths = {
    "main": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

metric = metrics["weighted_median"]
metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric
upcoming_metric_pct_chg = "upcoming_" + metric_pct_chg


In [31]:
core_path = dataset_paths["main"]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]


In [32]:
df = pd.read_csv(core_path)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
)

In [33]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change()

# set up target variables
df[upcoming_metric_pct_chg] = df.groupby(granularity_columns)[metric_pct_chg].shift(-1)

# time box
start_year = 2005
end_year = 2023

df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27728 entries, 18191 to 15439
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   year                                 27728 non-null  int64  
 1   area_code                            27728 non-null  int64  
 2   area                                 27728 non-null  object 
 3   unit_price_wmean                     27728 non-null  float64
 4   unit_price_wmedian                   27728 non-null  float64
 5   unit_price_mean                      27728 non-null  float64
 6   unit_price_median                    27728 non-null  float64
 7   total_traded_area                    27728 non-null  float64
 8   count                                27728 non-null  float64
 9   population                           25603 non-null  float64
 10  net_migration_ratio                  25603 non-null  float64
 11  taxable_income               

In [35]:
columns = [
    metric_pct_chg,
    metric,
    "year",
    "count",
    "total_traded_area",
    "population",
    "taxpayer_count",
    "taxable_income",
    "new_dwellings",
    "existing_dwellings",
    "net_migration_ratio",

    "area_code", # id
]

# add ratios and growths if necessary

df = df[columns]
df = df.dropna()


In [36]:
print(f"Initial Size: ", df.shape[0])
q = 0.01
filtered_df = df.copy()
filter_col = metric_pct_chg
filtered_df = filtered_df[
    (filtered_df[filter_col] >= filtered_df[filter_col].quantile(q))
    & (filtered_df[filter_col] <= filtered_df[filter_col].quantile(1 - q))
]
print(f"Filtered Size: ", filtered_df.shape[0])
filtered_df.describe()

Initial Size:  11365
Filtered Size:  11137


,unit_price_wmedian_pct_chg,unit_price_wmedian,year,count,total_traded_area,population,taxpayer_count,taxable_income,new_dwellings,existing_dwellings,net_migration_ratio,area_code
count,11137.000000,1.113700e+04,11137.000000,11137.000000,11137.000000,1.113700e+04,1.113700e+04,1.113700e+04,11137.000000,1.113700e+04,11137.000000,11137.000000
mean,0.051114,1.030075e+05,2014.494747,134.835683,33987.655114,1.481518e+05,6.675709e+04,2.270761e+08,1105.607255,6.975867e+04,-0.002375,21987.623058
std,0.422676,2.359960e+05,4.079049,259.943738,50153.308615,2.696482e+05,1.229808e+05,4.677415e+08,2477.181054,1.347166e+05,0.005458,12947.018475
min,-0.730657,5.103550e+02,2006.000000,1.000000,250.000000,2.961000e+03,1.124000e+03,2.670080e+06,0.000000,2.177000e+03,-0.058973,1100.000000
25%,-0.194490,2.019550e+04,2011.000000,29.000000,11440.000000,4.222600e+04,1.818800e+04,4.864604e+07,182.000000,1.850000e+04,-0.005622,11245.000000
50%,-0.011226,4.065087e+04,2015.000000,63.000000,19470.000000,7.033200e+04,3.080200e+04,9.327951e+07,411.000000,3.051800e+04,-0.002518,21209.000000
75%,0.190476,1.044896e+05,2018.000000,134.000000,35470.000000,1.408430e+05,6.405400e+04,2.058807e+08,964.000000,6.359900e+04,0.000453,32204.000000
max,2.389143,4.417898e+06,2021.000000,4358.000000,666150.000000,3.811873e+06,1.906224e+06,7.965148e+09,41746.000000,1.916062e+06,0.043446,47215.000000


In [37]:
filtered_df.to_csv(model_ready_data_paths["sequence"], index=False)